In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1642853,2021-09-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1642854,2021-09-11,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1642855,2021-09-12,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1642856,2021-09-13,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
42975,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
42977,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
42979,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
42981,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
42983,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1002198,2021-09-10,Arkansas,Arkansas,5001,3224,54.00,5000,Arkansas,AR,Arkansas,State,3017804
1002200,2021-09-11,Arkansas,Arkansas,5001,3242,54.00,5000,Arkansas,AR,Arkansas,State,3017804
1002202,2021-09-12,Arkansas,Arkansas,5001,3250,55.00,5000,Arkansas,AR,Arkansas,State,3017804
1002204,2021-09-13,Arkansas,Arkansas,5001,3255,55.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1642853,2021-09-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1642854,2021-09-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1642855,2021-09-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1642856,2021-09-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-09-14') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
602,2021-09-14,Snohomish,Washington,54850,668.00,53061,WA,County,822083,6672.08,81.26
1202,2021-09-14,Cook,Illinois,606266,11448.00,17031,IL,County,5150233,11771.62,222.28
1801,2021-09-14,Orange,California,312348,5316.00,6059,CA,County,3175692,9835.59,167.40
2399,2021-09-14,Maricopa,Arizona,668223,11075.00,4013,AZ,County,4485414,14897.69,246.91
2997,2021-09-14,Los Angeles,California,1435872,25713.00,6037,CA,County,10039107,14302.79,256.13
...,...,...,...,...,...,...,...,...,...,...,...
1641633,2021-09-14,Wheeler,Oregon,62,1.00,41069,OR,County,1332,4654.65,75.08
1641970,2021-09-14,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1642276,2021-09-14,Esmeralda,Nevada,46,0.00,32009,NV,County,873,5269.19,0.00
1642578,2021-09-14,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
602,2021-09-14,Snohomish,Washington,54850,668.00,53061,WA,County,822083,6672.08,81.26,81.26,6672.08
1202,2021-09-14,Cook,Illinois,606266,11448.00,17031,IL,County,5150233,11771.62,222.28,222.28,11771.62
1801,2021-09-14,Orange,California,312348,5316.00,6059,CA,County,3175692,9835.59,167.40,167.40,9835.59
2399,2021-09-14,Maricopa,Arizona,668223,11075.00,4013,AZ,County,4485414,14897.69,246.91,246.91,14897.69
2997,2021-09-14,Los Angeles,California,1435872,25713.00,6037,CA,County,10039107,14302.79,256.13,256.13,14302.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1641633,2021-09-14,Wheeler,Oregon,62,1.00,41069,OR,County,1332,4654.65,75.08,75.08,4654.65
1641970,2021-09-14,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1642276,2021-09-14,Esmeralda,Nevada,46,0.00,32009,NV,County,873,5269.19,0.00,0.00,5269.19
1642578,2021-09-14,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00,0.00,4733.73


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1048213,2021-09-14,Hale,Alabama,2815,81.00,1065,AL,County,14651,19213.71,552.86,552.86,19213.71,1
1141583,2021-09-14,Clarke,Alabama,4451,68.00,1025,AL,County,23622,18842.60,287.87,287.87,18842.60,2
725840,2021-09-14,Etowah,Alabama,18288,410.00,1055,AL,County,102268,17882.43,400.91,400.91,17882.43,3
608586,2021-09-14,Franklin,Alabama,5566,92.00,1059,AL,County,31362,17747.59,293.35,293.35,17747.59,4
726918,2021-09-14,Marshall,Alabama,16907,252.00,1095,AL,County,96774,17470.60,260.40,260.40,17470.60,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882027,2021-09-14,Johnson,Wyoming,995,13.00,56019,WY,County,8445,11782.12,153.94,153.94,11782.12,19
1387236,2021-09-14,Niobrara,Wyoming,258,4.00,56027,WY,County,2356,10950.76,169.78,169.78,10950.76,20
1245130,2021-09-14,Lincoln,Wyoming,2089,14.00,56023,WY,County,19830,10534.54,70.60,70.60,10534.54,21
1000601,2021-09-14,Sublette,Wyoming,1008,11.00,56035,WY,County,9831,10253.28,111.89,111.89,10253.28,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
819081,2021-09-14,Lowndes,Alabama,1674,55.00,1085,AL,County,9726,17211.60,565.49,565.49,17211.60,8,1
1048213,2021-09-14,Hale,Alabama,2815,81.00,1065,AL,County,14651,19213.71,552.86,552.86,19213.71,1,2
883638,2021-09-14,Greene,Alabama,1177,41.00,1063,AL,County,8111,14511.16,505.49,505.49,14511.16,47,3
347123,2021-09-14,Walker,Alabama,9987,302.00,1127,AL,County,63521,15722.36,475.43,475.43,15722.36,31,4
725301,2021-09-14,Dallas,Alabama,4558,173.00,1047,AL,County,37196,12254.01,465.10,465.10,12254.01,61,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213860,2021-09-14,Uinta,Wyoming,3248,21.00,56041,WY,County,20226,16058.54,103.83,103.83,16058.54,4,19
1561410,2021-09-14,Weston,Wyoming,845,7.00,56045,WY,County,6927,12198.64,101.05,101.05,12198.64,18,20
1245130,2021-09-14,Lincoln,Wyoming,2089,14.00,56023,WY,County,19830,10534.54,70.60,70.60,10534.54,21,21
815853,2021-09-14,Albany,Wyoming,5461,25.00,56001,WY,County,38880,14045.78,64.30,64.30,14045.78,9,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,81.26,6672.08,24,26
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,81.26,6672.08,24,26
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,81.26,6672.08,24,26
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,81.26,6672.08,24,26
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,81.26,6672.08,24,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1639038,2021-09-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1639039,2021-09-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1639040,2021-09-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1639041,2021-09-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
1043865,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,19213.71,2,1,1.00
1043866,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,19213.71,2,1,0.00
1043867,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,19213.71,2,1,0.00
1043868,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,19213.71,2,1,0.00
1043869,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,552.86,19213.71,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266174,2021-09-10,Crook,Wyoming,626,13.00,56011,WY,County,7584,8254.22,171.41,184.60,8662.97,9,23,7.00
1266175,2021-09-11,Crook,Wyoming,626,13.00,56011,WY,County,7584,8254.22,171.41,184.60,8662.97,9,23,0.00
1266176,2021-09-12,Crook,Wyoming,626,13.00,56011,WY,County,7584,8254.22,171.41,184.60,8662.97,9,23,0.00
1266177,2021-09-13,Crook,Wyoming,649,13.00,56011,WY,County,7584,8557.49,171.41,184.60,8662.97,9,23,23.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
815801,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,17211.60,1,8,1.00,0.00
815802,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,17211.60,1,8,0.00,0.00
815803,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,17211.60,1,8,0.00,0.00
815804,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,17211.60,1,8,0.00,0.00
815805,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,17211.60,1,8,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343280,2021-09-10,Teton,Wyoming,4611,11.00,56039,WY,County,23464,19651.38,46.88,46.88,19941.19,23,1,24.00,0.00
343281,2021-09-11,Teton,Wyoming,4611,11.00,56039,WY,County,23464,19651.38,46.88,46.88,19941.19,23,1,0.00,0.00
343282,2021-09-12,Teton,Wyoming,4611,11.00,56039,WY,County,23464,19651.38,46.88,46.88,19941.19,23,1,0.00,0.00
343283,2021-09-13,Teton,Wyoming,4655,11.00,56039,WY,County,23464,19838.90,46.88,46.88,19941.19,23,1,44.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-09-14') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
414671,2021-09-14,Imperial,California,33849,750.00,6025,CA,County,181215,18678.92,413.87,413.87,18678.92,1,3,12.00,0.00
2997,2021-09-14,Los Angeles,California,1435872,25713.00,6037,CA,County,10039107,14302.79,256.13,256.13,14302.79,2,9,1698.00,25.00
205939,2021-09-14,San Bernardino,California,343967,5441.00,6071,CA,County,2180085,15777.69,249.58,249.58,15777.69,3,4,564.00,2.00
91829,2021-09-14,Stanislaus,California,79230,1197.00,6099,CA,County,550660,14388.19,217.38,217.38,14388.19,4,7,306.00,1.00
727948,2021-09-14,Inyo,California,1611,39.00,6027,CA,County,18039,8930.65,216.20,216.20,8930.65,5,36,6.00,0.00
72995,2021-09-14,San Joaquin,California,95699,1581.00,6077,CA,County,762148,12556.49,207.44,207.44,12556.49,6,16,236.00,0.00
42417,2021-09-14,Riverside,California,354155,4851.00,6065,CA,County,2470546,14335.09,196.35,196.35,14335.09,7,8,544.00,14.00
659385,2021-09-14,Merced,California,38856,532.00,6047,CA,County,277680,13993.09,191.59,191.59,13993.09,8,10,417.00,5.00
113946,2021-09-14,Tulare,California,70927,887.00,6107,CA,County,466195,15214.02,190.26,190.26,15214.02,9,5,186.00,0.00
41303,2021-09-14,Fresno,California,133272,1853.00,6019,CA,County,999101,13339.19,185.47,185.47,13339.19,10,13,243.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1550428,2021-09-14,Lassen,California,6741,32.00,6035,CA,County,30573,22048.87,104.67,104.67,22048.87,32,1,138.00,1.00
883580,2021-09-14,Kings,California,29463,278.00,6031,CA,County,152940,19264.42,181.77,181.77,19264.42,11,2,94.00,4.00
414671,2021-09-14,Imperial,California,33849,750.00,6025,CA,County,181215,18678.92,413.87,413.87,18678.92,1,3,12.00,0.00
205939,2021-09-14,San Bernardino,California,343967,5441.00,6071,CA,County,2180085,15777.69,249.58,249.58,15777.69,3,4,564.00,2.00
113946,2021-09-14,Tulare,California,70927,887.00,6107,CA,County,466195,15214.02,190.26,190.26,15214.02,9,5,186.00,0.00
259137,2021-09-14,Kern,California,132607,1505.00,6029,CA,County,900202,14730.80,167.18,167.18,14730.80,14,6,482.00,4.00
91829,2021-09-14,Stanislaus,California,79230,1197.00,6099,CA,County,550660,14388.19,217.38,217.38,14388.19,4,7,306.00,1.00
42417,2021-09-14,Riverside,California,354155,4851.00,6065,CA,County,2470546,14335.09,196.35,196.35,14335.09,7,8,544.00,14.00
2997,2021-09-14,Los Angeles,California,1435872,25713.00,6037,CA,County,10039107,14302.79,256.13,256.13,14302.79,2,9,1698.00,25.00
659385,2021-09-14,Merced,California,38856,532.00,6047,CA,County,277680,13993.09,191.59,191.59,13993.09,8,10,417.00,5.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
414671,2021-09-14,Imperial,California,33849,750.00,6025,CA,County,181215,18678.92,413.87,413.87,18678.92,1,3,12.00,0.00
2997,2021-09-14,Los Angeles,California,1435872,25713.00,6037,CA,County,10039107,14302.79,256.13,256.13,14302.79,2,9,1698.00,25.00
205939,2021-09-14,San Bernardino,California,343967,5441.00,6071,CA,County,2180085,15777.69,249.58,249.58,15777.69,3,4,564.00,2.00
91829,2021-09-14,Stanislaus,California,79230,1197.00,6099,CA,County,550660,14388.19,217.38,217.38,14388.19,4,7,306.00,1.00
727948,2021-09-14,Inyo,California,1611,39.00,6027,CA,County,18039,8930.65,216.20,216.20,8930.65,5,36,6.00,0.00
72995,2021-09-14,San Joaquin,California,95699,1581.00,6077,CA,County,762148,12556.49,207.44,207.44,12556.49,6,16,236.00,0.00
42417,2021-09-14,Riverside,California,354155,4851.00,6065,CA,County,2470546,14335.09,196.35,196.35,14335.09,7,8,544.00,14.00
659385,2021-09-14,Merced,California,38856,532.00,6047,CA,County,277680,13993.09,191.59,191.59,13993.09,8,10,417.00,5.00
113946,2021-09-14,Tulare,California,70927,887.00,6107,CA,County,466195,15214.02,190.26,190.26,15214.02,9,5,186.00,0.00
41303,2021-09-14,Fresno,California,133272,1853.00,6019,CA,County,999101,13339.19,185.47,185.47,13339.19,10,13,243.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,3,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,3,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5452,09/10/21,Lassen,6577,31.00,21512.45,101.40,32,1,76.00,2.00
5453,09/11/21,Lassen,6577,31.00,21512.45,101.40,32,1,0.00,0.00
5454,09/12/21,Lassen,6577,31.00,21512.45,101.40,32,1,0.00,0.00
5455,09/13/21,Lassen,6603,31.00,21597.49,101.40,32,1,26.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,3,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,3,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5538,09/10/21,Fresno,131662,1838.00,13178.05,183.97,10,13,396.00,7.00
5539,09/11/21,Fresno,132281,1846.00,13240.00,184.77,10,13,619.00,8.00
5540,09/12/21,Fresno,132846,1853.00,13296.55,185.47,10,13,565.00,7.00
5541,09/13/21,Fresno,133029,1853.00,13314.87,185.47,10,13,183.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)